In [1]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt

In [154]:
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 5)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [3]:
manaus_shp = '/Users/morgansteckler/Downloads/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)

In [4]:
atto_shp = '/Users/morgansteckler/Downloads/venus_atto_footprint/venus_atto_footprint.shp'
atto = geemap.shp_to_ee(atto_shp)

In [5]:
corumba_shp = '/Users/morgansteckler/Downloads/venus_corumba_footprint/venus_corumba_footprint.shp'
corumba = geemap.shp_to_ee(corumba_shp)

In [6]:
mato_shp = '/Users/morgansteckler/Downloads/venus_mato_footprint/venus_mato_footprint.shp'
mato = geemap.shp_to_ee(mato_shp)

In [7]:
saop_shp = '/Users/morgansteckler/Downloads/venus_saop_footprint/venus_saop_footprint.shp'
saop = geemap.shp_to_ee(saop_shp)

In [185]:
#IMPORTANT
#Select geom of interest
fc = manaus

In [186]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int()
cirrusBitMask = ee.Number(2).pow(11).int()
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [187]:
#Load sentinel-2 data
s2 = (ee.ImageCollection('COPERNICUS/S2')#Select S2 1-C level data
      .select(['B5','B8', 'QA60']) #Select necessary bands to reduce memory usage
      .filterBounds(fc) #Narrow data spatially
      .filterDate(ee.Date('2016-01-01'), ee.Date('2019-12-31')) #Select years of interest
      .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
      .map(addNDRE)) #calculate NDRE and add as band

#Load copernicus land cover data
lc = (ee.Image("COPERNICUS/Landcover/100m/Proba-V/Global/2015") #select 2015
      .select('discrete_classification') #choose classification band
      .clip(fc)) #clip to geom

In [188]:
#Create a masked s2 collection
masked = s2.map(maskS2clouds)

In [189]:
#15-day step
years = ee.List.sequence(2016, 2019)
period = ee.List.sequence(1, 365, 15)
col = masked.select('NDRE')

def byYear(y):
    y = ee.Number(y)
    def ndreStepList(startDoy):
        startDoy = ee.Number(startDoy)
        return (col
                .filter(ee.Filter.calendarRange(y, y.add(1), 'year'))
                .filter(ee.Filter.calendarRange(startDoy, startDoy.add(15), 'day_of_year'))
                .qualityMosaic('NDRE')
               .set('day_of_year', [startDoy, y]))
    return period.map(ndreStepList)

mapped_doy = years.map(byYear).flatten()
col = ee.ImageCollection.fromImages(mapped_doy)

In [190]:
col = s2.map(lambda image: image.clip(fc)) #Reduce data to just area for analysis

In [144]:
info = col.aggregate_array('day_of_year').getInfo()
print(info)

[[1, 2016], [16, 2016], [31, 2016], [46, 2016], [61, 2016], [76, 2016], [91, 2016], [106, 2016], [121, 2016], [136, 2016], [151, 2016], [166, 2016], [181, 2016], [196, 2016], [211, 2016], [226, 2016], [241, 2016], [256, 2016], [271, 2016], [286, 2016], [301, 2016], [316, 2016], [331, 2016], [346, 2016], [361, 2016], [1, 2017], [16, 2017], [31, 2017], [46, 2017], [61, 2017], [76, 2017], [91, 2017], [106, 2017], [121, 2017], [136, 2017], [151, 2017], [166, 2017], [181, 2017], [196, 2017], [211, 2017], [226, 2017], [241, 2017], [256, 2017], [271, 2017], [286, 2017], [301, 2017], [316, 2017], [331, 2017], [346, 2017], [361, 2017], [1, 2018], [16, 2018], [31, 2018], [46, 2018], [61, 2018], [76, 2018], [91, 2018], [106, 2018], [121, 2018], [136, 2018], [151, 2018], [166, 2018], [181, 2018], [196, 2018], [211, 2018], [226, 2018], [241, 2018], [256, 2018], [271, 2018], [286, 2018], [301, 2018], [316, 2018], [331, 2018], [346, 2018], [361, 2018], [1, 2019], [16, 2019], [31, 2019], [46, 2019], [

In [220]:
an_img = col.first().select('NDRE')

In [224]:
#Manaus classes
shrub = lc.eq(20)
herb = lc.eq(30)
ag = lc.eq(40)
cl_ebl = lc.eq(112) #closed evergreen broad leaf
cl_forest = lc.eq(116)
op_ebl = lc.eq(122) #open evergreen broad leaf
op_forest = lc.eq(126)

In [225]:
def createVectors(classif):
    return classif.reduceToVectors(geometry=fc, scale=20, eightConnected=True, labelProperty='landcover')

In [226]:
shrub = createVectors(shrub)
herb = createVectors(herb)
ag = createVectors(ag)
cl_ebl = createVectors(cl_ebl)
cl_forest = createVectors(cl_forest)
op_ebl = createVectors(op_ebl)
op_forest = createVectors(op_forest)

In [227]:
shrub = shrub.filter(ee.Filter.equals('landcover',1))
herb = herb.filter(ee.Filter.equals('landcover',1))
ag = ag.filter(ee.Filter.equals('landcover',1))
cl_ebl = cl_ebl.filter(ee.Filter.equals('landcover',1))
cl_forest = cl_forest.filter(ee.Filter.equals('landcover',1))
op_ebl = op_ebl.filter(ee.Filter.equals('landcover',1))
op_forest = op_forest.filter(ee.Filter.equals('landcover',1))

In [228]:
shrubu = shrub.geometry()
herbu = herb.geometry()
agu = ag.geometry()
cl_eblu = cl_ebl.geometry()
cl_forestu = cl_forest.geometry()
op_eblu = op_ebl.geometry()
op_forestu = op_forest.geometry()

In [229]:
#Create an FC of a few classes
union = (ee.FeatureCollection([
    ee.Feature(shrubu, {'label':'Shrub'}),
    ee.Feature(herbu, {'label':'Forest'}),
    ee.Feature(agu, {'label':'Crop'}),
    ee.Feature(cl_eblu, {'label':'Closed EBL'}),
    ee.Feature(cl_forestu, {'label':'Closed Forest'}),
    ee.Feature(op_eblu, {'label':'Open EBL'}),
    ee.Feature(op_forestu, {'label':'Open Forest'})]))

In [231]:
#Make sure above worked
print(union.aggregate_array('label').getInfo())
Map.addLayer(union, {}, 'union')

['Shrub', 'Forest', 'Crop', 'Closed EBL', 'Closed Forest', 'Open EBL', 'Open Forest']


In [153]:
#Try Dr. Wu's zonal stats method and export as csv
#This isn't working. Maybe I can cast the values to an int rather than int8?
out_dir = "/Users/morgansteckler/Desktop/Test/"
geemap.zonal_statistics_by_group(an_img, union, out_dir, 'MEAN', scale=20)

The input image band must be integer type.


In [221]:
#Calculate mean of (something? NDRE?) for each class in the collection
def zonalStats(img):
    mean = ee.Reducer.mean()
    result = img.reduceRegions(collection=union, reducer=mean, scale=20)
    return result #Returns a feature collection

stats = zonalStats(an_img)

In [223]:
#This works, though I expect 7 values and only get 6...
#Also, I'm not sure what it's taking the mean of. Hopefully NDRE? But the values are quite small.
info = stats.aggregate_array('mean').getInfo()
print(info)

[0.005196842364966869, 0.0048230369575321674, 0.0034133470617234707, 0.0046998136676847935, 0.004713194910436869, 0.004642040468752384]
